In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!mkdir "/content/images"
!mkdir "/content/images/train"
!mkdir "/content/images/test"
!unrar x "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/train/train.rar" "/content/images/train"
!unrar x "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/test/test.rar" "/content/images/test"
!cp "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/train/annotations.json" "/content/images/train"
!cp "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/test/annotations.json" "/content/images/test"

mkdir: cannot create directory ‘/content/images’: File exists
mkdir: cannot create directory ‘/content/images/train’: File exists
mkdir: cannot create directory ‘/content/images/test’: File exists

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/train/train.rar


Would you like to replace the existing file /content/images/train/images/006316.jpg
 30041 bytes, modified on 2020-09-01 11:46
with a new one
 30041 bytes, modified on 2020-09-01 11:46

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit e

All OK

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/images/test/test.rar


Would you like to replace the existing file /content/images/test/images/006452.jpg
 33085 bytes, modified on 2020-09-01 09:47
with a new one
 33085 bytes, modified on 2020-09-01 09:47

In [7]:
import cv2
import numpy as np
import sys
from os import listdir
from os.path import isfile, join
from google.colab.patches import cv2_imshow
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import math

sys.path.append("/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/scripts/")
from data_generation import DataGeneration
from filter_cats import filtered_cats

In [8]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

%env SM_FRAMEWORK=tf.keras
!pip install segmentation_models
import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras


In [9]:
BACKBONE = 'efficientnetb5'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [10]:
SIZE_X = 128
SIZE_Y = 128
N_CATS = 16
ADD_BACKGROUND = True
N_CHANNELS = N_CATS + int(ADD_BACKGROUND)   # One more channel for background

INPUT_SHAPE = (SIZE_X, SIZE_Y, 3)
BATCH_SIZE = 32

TRAIN_IMAGES_PATH = "/content/images/train/images"
TRAIN_ANNOTATIONS_PATH = "/content/images/train/annotations.json"
TEST_IMAGES_PATH = "/content/images/test/images"
TEST_ANNOTATIONS_PATH = "/content/images/test/annotations.json"

In [11]:
def batch_generator(batchsize, images_path, annotation_path):
  print("Generator created for " + annotation_path)
  i_img = 0
  coco = COCO(annotation_path)

  categories_ids, categories_names, img_ids = filtered_cats(coco, n=N_CATS)
  np.random.shuffle(img_ids)

  images = coco.loadImgs(img_ids)
  img_paths = [img["file_name"] for img in images]

  data_gen = DataGeneration(coco, SIZE_X, SIZE_Y, categories_ids, add_background=ADD_BACKGROUND)

  while True:
    inputs = np.zeros((batchsize, SIZE_X, SIZE_Y, 3))
    outputs = np.zeros((batchsize, SIZE_X, SIZE_Y, N_CHANNELS))

    for i in range(batchsize):
      try:
        inputs[i] = data_gen.x_sample(join(images_path, img_paths[i_img]))
        outputs[i] = data_gen.y_sample(img_ids[i_img])
      except:
        i_img = (i_img + 1) % len(img_paths)
        inputs[i] = data_gen.x_sample(join(images_path, img_paths[i_img]))
        outputs[i] = data_gen.y_sample(img_ids[i_img])

      i_img = (i_img + 1) % len(img_paths)

    yield preprocess_input(inputs), outputs

In [12]:
def get_steps_per_epoch(batchsize, annotation_path):
  coco = COCO(annotation_path)
  categories_ids, categories_names, img_ids = filtered_cats(coco, n=N_CATS)

  return int(math.floor(len(img_ids) / batchsize))

In [13]:
model = sm.Unet(BACKBONE, input_shape=INPUT_SHAPE, encoder_weights='imagenet', classes=N_CHANNELS, activation='softmax', encoder_freeze=True)

total_loss = sm.losses.DiceLoss(per_image=True) + sm.losses.BinaryFocalLoss()
metrics = [sm.metrics.IOUScore(threshold=0.5, per_image=True)]

model.compile(optimizer='Adam', loss=total_loss, metrics=metrics)
model.summary()

115515392/115515256 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 64, 64, 48)   1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 64, 64, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 64, 64, 48)   0           stem_bn[0][0]                    
___________________________

In [18]:
from tensorflow.keras.utils import plot_model
plot_model(model, "/content/model_graph.png",show_shapes=True)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
callbacks = [
             ModelCheckpoint("/content/logs/model_checkpoints/food_segmentation.h5", verbose=1, save_best_only=True),
             EarlyStopping(patience=5, monitor="val_loss"),
             TensorBoard(log_dir="/content/logs/tensorboard_logs")
]

gen_train = batch_generator(BATCH_SIZE, TRAIN_IMAGES_PATH, TRAIN_ANNOTATIONS_PATH)
gen_val = batch_generator(BATCH_SIZE, TEST_IMAGES_PATH, TEST_ANNOTATIONS_PATH)

In [ ]:
history = model.fit(gen_train,
                    verbose=1,
                    steps_per_epoch=get_steps_per_epoch(BATCH_SIZE, TRAIN_ANNOTATIONS_PATH),
                    epochs=20,
                    validation_data=gen_val,
                    validation_steps=get_steps_per_epoch(BATCH_SIZE, TEST_ANNOTATIONS_PATH),
                    #class_weight=class_weights,
                    shuffle=True,
                    callbacks=callbacks)

loading annotations into memory...
Done (t=1.78s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
Generator created for /content/images/train/annotations.json
loading annotations into memory...
Done (t=1.45s)
creating index...
index created!
Epoch 1/20
417/417 [==============================] - ETA: 0s - loss: 0.9503 - iou_score: 0.8594Generator created for /content/images/test/annotations.json
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
417/417 [==============================] - 294s 577ms/step - loss: 0.9503 - iou_score: 0.8593 - val_loss: 0.9569 - val_iou_score: 0.7232

Epoch 00001: val_loss improved from inf to 0.95694, saving model to /content/logs/model_checkpoints/food_segmentation.h5
Epoch 2/20
417/417 [==============================] - 229s 551ms/step - loss: 0.9215 - iou_score: 0.8216 - val_loss: 0.9510 - val_iou_score: 0.7642

Epoch 00002: val_loss improved from 0.95694

In [ ]:
!cp "/content/logs/model_checkpoints/food_segmentation.h5" "/content/drive/Shareddrives/Intercambio/MATERIAS/Deep Learning/Food Recognition/weights/seg_models_pretrained_facu.h5"